# County Level COVID-19 Reports
### Specify the name of the state in the next box

In [ ]:
#s = "Massachusetts"
s = "Florida"
#s = "New York"
#s = "Georgia"
#s = "Tennessee"
#s = "North Carolina"

s='Iowa'
s = 'Nebraska'
s = "North Dakota"
s = "South Dakota"
s = "Virginia"

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime
from datetime import date
from datetime import timedelta


from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
display(HTML("<style>div.output_scroll { height: 65em; }</style>"))
now = (str(datetime.now()))
nowdate = now[0:10] 
print(nowdate)

import matplotlib.ticker
class MyLocator(matplotlib.ticker.AutoMinorLocator):
    def __init__(self, n=14):
        super().__init__(n=n)
matplotlib.ticker.AutoMinorLocator = MyLocator        



# Read in the time series files published by Johns Hopkins
### There are many quality issues with this data.  The column names were changed in the 3/23 dataset.  Then on 3/25, they retroactively made changes to the 3/22 dataset.  Values in Province/State are inconsistent in the US data.  At times they have a City, ST.  Other times the name of the State is spelled out completely.  This needs to be cleaned up.  DAW 3-25-2020

### Found a coding error that caused the 3/22 data to be read and appended 3 times but no 3/23 or 3/24 data was read.  Error fixed.  DAW 3-25-2020
### 3/22 Data is first in this set of tables that contains the Admin2 field which has the County in it for the US.

In [ ]:
tshx1 = "https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv"
tshx2 = "https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv"
tshx3 = "https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv"
# READ CONFIRMED COUNTS
c = pd.read_csv(tshx1, error_bad_lines=False)
c = c[c["Province_State"] == s]
c = c.drop(['UID',"iso2","iso3","code3","FIPS","Lat","Long_","Combined_Key","Country_Region" ], axis = 1, inplace=False)
c = c[(c["Admin2"]!="Bronx") & (c["Admin2"]!="Queens")& (c["Admin2"]!="Kings")& (c["Admin2"]!="Richmond")& (c["Admin2"]!="Unassigned") & (c["Admin2"]!="Out of NY")]
# READ DEATH COUNTS
d = pd.read_csv(tshx2, error_bad_lines=False)
d = d[d["Province_State"] == s]
d = d.drop(['UID',"iso2","iso3","code3","FIPS","Lat","Long_","Combined_Key","Country_Region" ,"Population"], axis = 1, inplace=False)
d = d[(d["Admin2"]!="Bronx") & (d["Admin2"]!="Queens")& (d["Admin2"]!="Kings")& (d["Admin2"]!="Richmond")& (d["Admin2"]!="Unassigned") & (d["Admin2"]!="Out of NY")]
 

In [ ]:
c2=(c.set_index(["Admin2",'Province_State'])
   .stack()
   .rename_axis(['Admin2','Province/State', 'Date'])
   .reset_index(name='Confirmed')
)

c2.set_index(['Admin2','Province/State','Date'])
c2["Date"] = pd.to_datetime(c2['Date'])
c2["ConfDelta"] = c2["Confirmed"].diff()
#c2 = c2.loc[(c2['ConfDelta'] < 0),'test']=0
# Set first period equal to 0 - would be negative or NaN based on the structure of the data.
c2.loc[c2['ConfDelta'] < 0, 'ConfDelta'] = 0
c2 = c2.fillna(0)


In [ ]:
d2=(d.set_index(["Admin2",'Province_State'])
   .stack()
   .rename_axis(['Admin2','Province/State', 'Date'])
   .reset_index(name='Deaths')
)

d2.set_index(['Admin2','Province/State','Date'])
d2["Date"] = pd.to_datetime(d2['Date'])
d2["DeathDelta"] = d2["Deaths"].diff()

# Set first period equal to 0 - would be negative or NaN based on the structure of the data.
d2.loc[d2['DeathDelta'] < 0, 'DeathDelta'] = 0
d2 = d2.fillna(0)


In [ ]:
#State data
sd = c2.merge(d2, on=['Province/State','Admin2','Date'])
sd['Admin2'] =sd['Admin2'].str.replace('.','')
sd = sd[sd['Admin2']!="Out of FL"]

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>div.output_scroll { height: 65em; }</style>"))

# Add Population Data

In [ ]:
#Population
#USpop = pd.read_csv("G://Data//SARS-CoV-2//US_POP_2018.csv").fillna(0).rename(columns={"STATE ABBREV": "state"})
#USpop.head()
# County Population data from US Census Bureau
#popin = "G://Data//SARS-CoV-2//USCensus_County_Pop_2019.csv"
# https://github.com/DavidAWest/COVID_Coding/raw/master/USCensus_County_Pop_2019.csv
popin = "https://github.com/DavidAWest/COVID_Coding/raw/master/USCensus_County_Pop_2019.csv"
cpop = pd.read_csv(popin).fillna(0).replace({"County_state" : {"County" : "", "." : ""}})#.rename(columns={"County_state": "Admin2"})
cpop['Admin2'] = cpop['County_state'].str.replace(' County', '')
cpop['Admin2'] =cpop['Admin2'].str.replace('.','')
#cpop['pos'] = cpop["Admin2"].str.find(",") 
cpop['Admin2'] = cpop["Admin2"].str.split(',').str[0]
cpop['Province/State'] = cpop["County_state"].str.split(',').str[1].str.lstrip()
cpop['Province/State'].unique()
cpop = cpop[cpop['Province/State'] == s]

In [ ]:
# County Land Area from US CENSUS BUREAU https://www2.census.gov/library/publications/2011/compendia/usa-counties/excel/LND01.xls
# https://www2.census.gov/library/publications/2011/compendia/usa-counties/excel/LND01.xls
# https://www.census.gov/library/publications/2011/compendia/usa-counties-2011.html#LND
# https://www.census.gov/library/publications/2011/compendia/usa-counties-2011/file-layout.html
# LND110200D = Land Area in sq mi
# LND210200D = Water Area in sq mi
# LND010200D = Total Area in sq mi
# STCOU = Code column header - FIPS State and County Code stored in this column (Excel spreadhseet column B) from rows 2 to 3197.
# Areaname = UNITED STATES | ALABAMA | Autauga, AL
from pandas import DataFrame
#lndin = "G://Data//SARS-CoV-2//Census 2012 County Data Land Area.csv"
lndin = "https://github.com/DavidAWest/COVID_Coding/raw/master/Census%202012%20County%20Data%20Land%20Area.csv"
lnd = pd.read_csv(lndin,usecols=['Areaname','STCOU','LND110200D']).fillna(0)# .replace({"County_state" : {"County" : "", "." : ""}})#.rename(columns={"County_state": "Admin2"})
x = lnd.Areaname.unique().tolist()
xdf = DataFrame(x ,columns=["Areaname"]) #.toframe().to_csv('CensusAreaName.csv', index=False)
xdf.to_csv('CensusAreaName.csv',index=False)
#stnamesfile = "G://Data//SARS-CoV-2//CensusAreaName.csv"
stnamesfile = "https://github.com/DavidAWest/COVID_Coding/raw/master/CensusAreaName.csv"
stname = pd.read_csv(stnamesfile, error_bad_lines=False, usecols = ['Areaname','Admin2','StateAbrev','State'])
stname

In [ ]:
sd = sd.merge(cpop, on=["Province/State","Admin2","Province/State"])
sd.head()

# Create new columns for analysis

In [ ]:
sd['Conf_per_10K'] = 10000* sd['Confirmed'] / sd['POP_2019']
sd['Conf_inc_per_10K'] = 10000 * sd['ConfDelta']/sd['POP_2019']
sd['Deaths_per_10K'] = 10000 * sd['Deaths']/sd['POP_2019']
countylist = sd[sd['Date']==sd['Date'].max()].sort_values(by=['Conf_per_10K'])
countylist.head()

In [ ]:
# list of counties in order of their Confirmed Cases Per 10K
counties = countylist['Admin2'].unique()


# Per Capita (10K) Plots

In [ ]:
# PLOT 3,5 and 7 day moving averages

import itertools
y= 0.20
colors = itertools.cycle(["maroon","dodgerblue","darkblue","olive"  ,"red","aqua","goldenrod","blue","tomato","magenta","palegreen","deepskyblue","deeppink","darkorchid","turquoise", "indigo",
                          "violet","cadetblue","chartreuse","saddlebrown"])

#states["5 day moving"] = states["positiveIncrease"].rolling(window=5).mean()
#states["3 day moving"] = states["positiveIncrease"].rolling(window=3).mean()
#states["7 day moving"] = states["positiveIncrease"].rolling(window=7).mean()


#states = states.merge(demog,on = ["state"])

fig=plt.figure(figsize=(20, 8), dpi= 80, facecolor='w', edgecolor='k')#.legend( prop={'size': 18})
#moving = df.rolling(window=5)['MA'].mean()
#fig=plt.figure(figsize=(20, 12), dpi= 80, facecolor='w', edgecolor='k').legend( prop={'size': 18})
y= 0.20
x= 0.1
for county in counties:
    cd = sd[sd['Admin2']==county].copy().fillna(0)


    
    c=next(colors)
    plt.plot( 'Date', 'Conf_per_10K',data=cd, marker='', markerfacecolor=c, markersize=12, color=c,linewidth=4)
    plt.annotate( county + ": "+ cd["Conf_per_10K"].iloc[(-1)].round(3).astype(str),  color=c,size=14, 
                 xy=(x, y), 
                 xytext=(x, y),horizontalalignment = 'left',
                 xycoords = 'figure fraction',textcoords='figure fraction')
    y+=0.023
    
    if y > 0.9:
        y = 0.20
        x += 0.1

title = "COVID-19 Infections Per Capita (per 10,000) State of " + s 

    #plt.annotate(s +" 3-day: "+ states[states["state"]==s ]["3 day moving"].iloc[(-1)].round(3).astype(str),  color=c,size=20, 
    #             xy=(0.15, y), 
    #             xytext=(0.15, y),horizontalalignment = 'left',
    #             xycoords = 'figure fraction',textcoords='figure fraction')

plt.title(title, fontsize=24)
plt.xlabel('Date', fontsize=20)
plt.ylabel('Count of Cases COVID-19 Per 10K', fontsize=24)
plt.rcParams['xtick.labelsize'] =16
plt.figtext(0.90, 0.01, 'Data gathered by Johns Hopkins University https://github.com/CSSEGISandData ' + now, horizontalalignment='right')
plt.figtext(0.10, 0.01, 'David A. West, IBM')
plt.tick_params(axis='both', which='minor', labelsize=16)
plt.tick_params(axis='both', which='major', labelsize=16)
plt.grid(which='major', axis='both', linestyle='--',linewidth='2')
plt.grid(which='minor', axis='both', linestyle=':', linewidth='2', color='white')
plt.xticks(fontsize=14, rotation=0)
matplotlib.ticker.AutoMinorLocator(n=2)
plt.minorticks_on()
plt.grid( b=True,which='minor', axis='x', color='black', linestyle='-',alpha=0.1)
fn = s + '_counties_inf_rate_' + nowdate +'.jpg'
plt.savefig(fn)

plt.show()

# County Level Charts - Infection Rates and Moving Averages

In [ ]:
# PLOT 3,5 and 7 day moving averages

import itertools
y= 0.20
colors = itertools.cycle(["maroon","dodgerblue","darkblue","olive" ]) #,
#                          "red","aqua","goldenrod","blue",
#                          "tomato","magenta","palegreen","deepskyblue",
#                          "deeppink","darkorchid","turquoise", "indigo",
#                          "violet","cadetblue","chartreuse","saddlebrown"])

#states["5 day moving"] = states["positiveIncrease"].rolling(window=5).mean()
#states["3 day moving"] = states["positiveIncrease"].rolling(window=3).mean()
#states["7 day moving"] = states["positiveIncrease"].rolling(window=7).mean()


#states = states.merge(demog,on = ["state"])


#moving = df.rolling(window=5)['MA'].mean()
#fig=plt.figure(figsize=(20, 12), dpi= 80, facecolor='w', edgecolor='k').legend( prop={'size': 18})

for county in counties:
    cd = sd[sd['Admin2']==county].copy().fillna(0)
    cd["3 day moving PC"] = cd["Conf_inc_per_10K"].rolling(window=3).mean()
    cd["5 day moving PC"] = cd["Conf_inc_per_10K"].rolling(window=5).mean()
    cd["7 day moving PC"] = cd["Conf_inc_per_10K"].rolling(window=7).mean()
    if cd["3 day moving PC"].iloc[(-2)] != 0:
        dt3PC = (100*(cd["3 day moving PC"].iloc[(-1)]-cd["3 day moving PC"].iloc[(-2)])/cd["3 day moving PC"].iloc[(-2)]).round(3).astype(str)
    else:
        dt3PC = "0"
    if cd["5 day moving PC"].iloc[(-2)] != 0:
        dt5PC = (100*(cd["5 day moving PC"].iloc[(-1)]-cd["5 day moving PC"].iloc[(-2)])/cd["5 day moving PC"].iloc[(-2)]).round(3).astype(str)
    else:
        dt5PC = "0"
    if cd["7 day moving PC"].iloc[(-2)] != 0:
        dt7PC = (100*(cd["7 day moving PC"].iloc[(-1)]-cd["7 day moving PC"].iloc[(-2)])/cd["7 day moving PC"].iloc[(-2)]).round(3).astype(str)
    else:
        dt7PC = "0"
    two_week_changePC = cd["7 day moving PC"].iloc[(-1)] - cd["7 day moving PC"].iloc[(-14)] 

    

       
#states["3 day moving"] = states["positiveIncrease"].rolling(window=3).mean()
#states["7 day moving"] = states["positiveIncrease"].rolling(window=7).mean()

    y= 0.30
    
    fig=plt.figure(figsize=(20, 8), dpi= 80, facecolor='w', edgecolor='k')#.legend( prop={'size': 18})
    
    c=next(colors)
    plt.plot( 'Date', 'Conf_inc_per_10K',data=cd, marker='', markerfacecolor=c, markersize=12, color=c,linewidth=4)
    plt.annotate("Daily Increase: "+ cd["Conf_inc_per_10K"].iloc[(-1)].round(3).astype(str),  color=c,size=20, 
                 xy=(0.15, y), 
                 xytext=(0.15, y),horizontalalignment = 'left',
                 xycoords = 'figure fraction',textcoords='figure fraction')
    c=next(colors)
    y+=0.035
    plt.plot( 'Date', "3 day moving PC", data = cd , marker='', markerfacecolor=c, markersize=12, color=c,linewidth=4)
    plt.annotate("3 day moving average: "+ cd["3 day moving PC"].iloc[(-1)].round(3).astype(str) + " (Change: " + dt3PC +"% )",  color=c,size=20, 
                 xy=(0.15, y), 
                 xytext=(0.15, y),horizontalalignment = 'left',
                 xycoords = 'figure fraction',textcoords='figure fraction')

    
    c=next(colors)
    plt.plot( 'Date', "5 day moving PC", data = cd , marker='', markerfacecolor=c, markersize=12, color=c,linewidth=4)
    y+=0.035
    plt.annotate("5 day moving average: "+ cd["5 day moving PC"].iloc[(-1)].round(3).astype(str) + " (Change: " + dt5PC +"% )",  color=c,size=20, 
                 xy=(0.15, y), 
                 xytext=(0.15, y),horizontalalignment = 'left',
                 xycoords = 'figure fraction',textcoords='figure fraction')
    c=next(colors)
    plt.plot( 'Date', "7 day moving PC", data = cd , marker='', markerfacecolor=c, markersize=12, color=c,linewidth=4)
    y+=0.035
    plt.annotate("7 day moving average: "+ cd["7 day moving PC"].iloc[(-1)].round(3).astype(str) + " (Change: " + dt7PC +"% )",  color=c,size=20, 
                 xy=(0.15, y), 
                 xytext=(0.15, y),horizontalalignment = 'left',
                 xycoords = 'figure fraction',textcoords='figure fraction')
    y+=0.045
    plt.annotate("2 Week Change to 7-day moving: "+ two_week_changePC.round(3).astype(str) ,  color='black' ,size=20, 
                 xy=(0.15, y), 
                 xytext=(0.15, y),horizontalalignment = 'left',
                 xycoords = 'figure fraction',textcoords='figure fraction')

    
    
    

    title = "COVID-19 Daily Positive Infections Per Capita (per 10,000): " + county + " County," + s 

    #plt.annotate(s +" 3-day: "+ states[states["state"]==s ]["3 day moving"].iloc[(-1)].round(3).astype(str),  color=c,size=20, 
    #             xy=(0.15, y), 
    #             xytext=(0.15, y),horizontalalignment = 'left',
    #             xycoords = 'figure fraction',textcoords='figure fraction')
#    y+=0.025
    plt.title(title, fontsize=24)
    plt.xlabel('Date', fontsize=20)
    plt.ylabel('Daily New Cases of COVID-19 Per 10K', fontsize=24)
    plt.rcParams['xtick.labelsize'] =22
    plt.figtext(0.90, 0.01, 'Data gathered by Johns Hopkins University https://github.com/CSSEGISandData ' + now, horizontalalignment='right')
    plt.figtext(0.10, 0.01, 'David A. West, IBM')
    plt.tick_params(axis='both', which='minor', labelsize=22)
    plt.tick_params(axis='both', which='major', labelsize=22)
    plt.grid(which='major', axis='both', linestyle='solid',linewidth='3')
    plt.grid(which='minor', axis='both', linestyle=':', linewidth='2', color='white')
    plt.xticks(fontsize=14, rotation=0)
    matplotlib.ticker.AutoMinorLocator(n=2)
    plt.minorticks_on()
    plt.grid( b=True,which='minor', axis='x', color='black', linestyle='-',alpha=0.1)
    fn = county + '_' + s + '_counties_inf_rate_' + nowdate +'.jpg'
    plt.savefig(fn)

    plt.show()

# PLOTS - COUNTY LEVEL - COUNTS and Moving Averages

In [ ]:
# PLOT 3,5 and 7 day moving averages

import itertools
y= 0.20
colors = itertools.cycle(["maroon","dodgerblue","darkblue","olive" ]) #,
#                          "red","aqua","goldenrod","blue",
#                          "tomato","magenta","palegreen","deepskyblue",
#                          "deeppink","darkorchid","turquoise", "indigo",
#                          "violet","cadetblue","chartreuse","saddlebrown"])

#states["5 day moving"] = states["positiveIncrease"].rolling(window=5).mean()
#states["3 day moving"] = states["positiveIncrease"].rolling(window=3).mean()
#states["7 day moving"] = states["positiveIncrease"].rolling(window=7).mean()


#states = states.merge(demog,on = ["state"])


#moving = df.rolling(window=5)['MA'].mean()
#fig=plt.figure(figsize=(20, 12), dpi= 80, facecolor='w', edgecolor='k').legend( prop={'size': 18})

for county in counties:
    cd = sd[sd['Admin2']==county].copy().fillna(0)
    cd["3 day moving"] = cd["ConfDelta"].rolling(window=3).mean()
    cd["5 day moving"] = cd["ConfDelta"].rolling(window=5).mean()
    cd["7 day moving"] = cd["ConfDelta"].rolling(window=7).mean()
    if cd["3 day moving"].iloc[(-2)] != 0:
        dt3 = (100*(cd["3 day moving"].iloc[(-1)]-cd["3 day moving"].iloc[(-2)])/cd["3 day moving"].iloc[(-2)]).round(3).astype(str)
    else:
        dt3 = "0"
    if cd["5 day moving"].iloc[(-2)] != 0:
        dt5 = (100*(cd["5 day moving"].iloc[(-1)]-cd["5 day moving"].iloc[(-2)])/cd["5 day moving"].iloc[(-2)]).round(3).astype(str)
    else:
        dt5 = "0"
    if cd["7 day moving"].iloc[(-2)] != 0:
        dt7 = (100*(cd["7 day moving"].iloc[(-1)]-cd["7 day moving"].iloc[(-2)])/cd["7 day moving"].iloc[(-2)]).round(3).astype(str)
    else:
        dt7 = "0"
    two_week_change = cd["7 day moving"].iloc[(-1)] - cd["7 day moving"].iloc[(-14)] 

    

       
#states["3 day moving"] = states["positiveIncrease"].rolling(window=3).mean()
#states["7 day moving"] = states["positiveIncrease"].rolling(window=7).mean()

    y= 0.30
    
    fig=plt.figure(figsize=(20, 8), dpi= 80, facecolor='w', edgecolor='k')#.legend( prop={'size': 18})
    
    c=next(colors)
    plt.plot( 'Date', 'ConfDelta',data=cd, marker='', markerfacecolor=c, markersize=12, color=c,linewidth=4)
    plt.annotate("Daily Increase: "+ cd["ConfDelta"].iloc[(-1)].round(3).astype(str),  color=c,size=20, 
                 xy=(0.15, y), 
                 xytext=(0.15, y),horizontalalignment = 'left',
                 xycoords = 'figure fraction',textcoords='figure fraction')
    c=next(colors)
    y+=0.035
    plt.plot( 'Date', "3 day moving", data = cd , marker='', markerfacecolor=c, markersize=12, color=c,linewidth=4)
    plt.annotate("3 day moving average: "+ cd["3 day moving"].iloc[(-1)].round(3).astype(str) + " (Change: " + dt3 +"% )",  color=c,size=20, 
                 xy=(0.15, y), 
                 xytext=(0.15, y),horizontalalignment = 'left',
                 xycoords = 'figure fraction',textcoords='figure fraction')

    
    c=next(colors)
    plt.plot( 'Date', "5 day moving", data = cd , marker='', markerfacecolor=c, markersize=12, color=c,linewidth=4)
    y+=0.035
    plt.annotate("5 day moving average: "+ cd["5 day moving"].iloc[(-1)].round(3).astype(str) + " (Change: " + dt5 +"% )",  color=c,size=20, 
                 xy=(0.15, y), 
                 xytext=(0.15, y),horizontalalignment = 'left',
                 xycoords = 'figure fraction',textcoords='figure fraction')
    c=next(colors)
    plt.plot( 'Date', "7 day moving", data = cd , marker='', markerfacecolor=c, markersize=12, color=c,linewidth=4)
    y+=0.035
    plt.annotate("7 day moving average: "+ cd["7 day moving"].iloc[(-1)].round(3).astype(str) + " (Change: " + dt7 +"% )",  color=c,size=20, 
                 xy=(0.15, y), 
                 xytext=(0.15, y),horizontalalignment = 'left',
                 xycoords = 'figure fraction',textcoords='figure fraction')
    y+=0.045
    plt.annotate("2 Week Change to 7-day moving: "+ two_week_change.round(3).astype(str) ,  color='black' ,size=20, 
                 xy=(0.15, y), 
                 xytext=(0.15, y),horizontalalignment = 'left',
                 xycoords = 'figure fraction',textcoords='figure fraction')

    
    
    

    title = "COVID-19 Daily Positive Infections: " + county + " County," + s 

    #plt.annotate(s +" 3-day: "+ states[states["state"]==s ]["3 day moving"].iloc[(-1)].round(3).astype(str),  color=c,size=20, 
    #             xy=(0.15, y), 
    #             xytext=(0.15, y),horizontalalignment = 'left',
    #             xycoords = 'figure fraction',textcoords='figure fraction')
#    y+=0.025
    plt.title(title, fontsize=24)
    plt.xlabel('Date', fontsize=20)
    plt.ylabel('Daily New Cases of COVID-19', fontsize=24)
    plt.rcParams['xtick.labelsize'] =22
    plt.figtext(0.90, 0.01, 'Data gathered by Johns Hopkins University https://github.com/CSSEGISandData ' + now, horizontalalignment='right')
    plt.figtext(0.10, 0.01, 'David A. West, IBM')
    plt.tick_params(axis='both', which='minor', labelsize=22)
    plt.tick_params(axis='both', which='major', labelsize=22)
    plt.grid(which='major', axis='both', linestyle='--',linewidth='2')
    plt.grid(which='minor', axis='both', linestyle=':', linewidth='2', color='white')
    plt.xticks(fontsize=14, rotation=0)
    
    matplotlib.ticker.AutoMinorLocator(n=2)
    plt.minorticks_on()
    plt.grid( b=True,which='minor', axis='x', color='black', linestyle='-',alpha=0.1)
    fn = s + '-' + county + '_County_case_count_' + nowdate +'.jpg'
    plt.savefig(fn)

    plt.show()
    

In [ ]:
sd2 = sd.groupby(["Province/State","Date"])['Confirmed', 'ConfDelta', 'Deaths', 'DeathDelta'].sum().reset_index()
sd2

# State Level Plot

In [ ]:
# PLOT 3,5 and 7 day moving averages
import itertools
y= 0.20
colors = itertools.cycle(["maroon","dodgerblue","red","darkorange","yellowgreen","gold","olive","indianred",
                          "green","turquoise","deepskyblue","darkorchid", "indigo","violet","deeppink"])

#states["5 day moving"] = states["positiveIncrease"].rolling(window=5).mean()
#states["3 day moving"] = states["positiveIncrease"].rolling(window=3).mean()
#states["7 day moving"] = states["positiveIncrease"].rolling(window=7).mean()


#states = states.merge(demog,on = ["state"])


#moving = df.rolling(window=5)['MA'].mean()
#fig=plt.figure(figsize=(20, 12), dpi= 80, facecolor='w', edgecolor='k').legend( prop={'size': 18})

sd2["3 day moving"] = sd2["ConfDelta"].rolling(window=3).mean()
sd2["5 day moving"] = sd2["ConfDelta"].rolling(window=5).mean()
sd2["7 day moving"] = sd2["ConfDelta"].rolling(window=7).mean()
#states["3 day moving"] = states["positiveIncrease"].rolling(window=3).mean()
#states["7 day moving"] = states["positiveIncrease"].rolling(window=7).mean()
if sd2["3 day moving"].iloc[(-2)] != 0:
    dt3 = (100*(sd2["3 day moving"].iloc[(-1)]-sd2["3 day moving"].iloc[(-2)])/sd2["3 day moving"].iloc[(-2)]).round(3).astype(str)
else:
    dt3 = "0"
if sd2["5 day moving"].iloc[(-2)] != 0:
    dt5 = (100*(sd2["5 day moving"].iloc[(-1)]-sd2["5 day moving"].iloc[(-2)])/sd2["5 day moving"].iloc[(-2)]).round(3).astype(str)
else:
    dt5 = "0"
if sd2["7 day moving"].iloc[(-2)] != 0:
    dt7 = (100*(sd2["7 day moving"].iloc[(-1)]-sd2["7 day moving"].iloc[(-2)])/sd2["7 day moving"].iloc[(-2)]).round(3).astype(str)
else:
    dt7 = "0"
    
cfr = (100*sd2["Deaths"].iloc[(-1)] / sd2["Confirmed"]).iloc[(-1)].round(1).astype(str)
cases = sd2["Confirmed"].iloc[(-1)].astype(str)
dead = sd2["Deaths"].iloc[(-1)].astype(str)

two_week_change = sd2["7 day moving"].iloc[(-1)] - sd2["7 day moving"].iloc[(-14)] 


y= 0.30

fig=plt.figure(figsize=(20, 8), dpi= 80, facecolor='w', edgecolor='k')#.legend( prop={'size': 18})

c=next(colors)
plt.plot( 'Date', 'ConfDelta',data=sd2, marker='', markerfacecolor=c, markersize=12, color=c,linewidth=4)
plt.annotate("Daily Increase: "+ sd2["ConfDelta"].iloc[(-1)].round(3).astype(str),  color=c,size=20, 
             xy=(0.15, y), 
             xytext=(0.15, y),horizontalalignment = 'left',
             xycoords = 'figure fraction',textcoords='figure fraction')

c=next(colors)
plt.plot( 'Date', "3 day moving", data = sd2 , marker='', markerfacecolor=c, markersize=12, color=c,linewidth=4)
y+=0.035
plt.annotate("3 day moving average: "+ sd2["3 day moving"].iloc[(-1)].round(3).astype(str) + " (Change: " + dt3 +"% )",  color=c,size=20, 
             xy=(0.15, y), 
             xytext=(0.15, y),horizontalalignment = 'left',
             xycoords = 'figure fraction',textcoords='figure fraction')


c=next(colors)


plt.plot( 'Date', "5 day moving", data = sd2 , marker='', markerfacecolor=c, markersize=12, color=c,linewidth=4)
c=next(colors)
y+=0.035
plt.annotate("5 day moving average: "+ sd2["5 day moving"].iloc[(-1)].round(3).astype(str) + " (Change: " + dt5 +"% )",  color=c,size=20, 
             xy=(0.15, y), 
             xytext=(0.15, y),horizontalalignment = 'left',
             xycoords = 'figure fraction',textcoords='figure fraction')


plt.plot( 'Date', "7 day moving", data = sd2 , marker='', markerfacecolor=c, markersize=12, color=c,linewidth=4)
y+=0.035
plt.annotate("7 day moving average: "+ sd2["7 day moving"].iloc[(-1)].round(3).astype(str) + " (Change: " + dt7 +"% )",  color=c,size=20, 
             xy=(0.15, y), 
             xytext=(0.15, y),horizontalalignment = 'left',
             xycoords = 'figure fraction',textcoords='figure fraction')


y+=0.045
plt.annotate("2 Week Change to 7-day moving: "+ two_week_change.round(3).astype(str) ,  color='black' ,size=20, 
             xy=(0.15, y), 
             xytext=(0.15, y),horizontalalignment = 'left',
             xycoords = 'figure fraction',textcoords='figure fraction')
y+=0.10
plt.annotate("Total Confirmed Cases: " + cases , color = 'black', size = 20, xy=(0.15, y), 
             xytext=(0.15, y),horizontalalignment = 'left',
             xycoords = 'figure fraction',textcoords='figure fraction')
y += 0.10
plt.annotate("Total Deaths: " + dead , color = 'black', size = 20, xy=(0.15, y), 
             xytext=(0.15, y),horizontalalignment = 'left',
             xycoords = 'figure fraction',textcoords='figure fraction')
y += 0.10
plt.annotate("Case Fatality Rate: " + cfr +"%", color = 'black', size = 20, xy=(0.15, y), 
             xytext=(0.15, y),horizontalalignment = 'left',
             xycoords = 'figure fraction',textcoords='figure fraction')

title = "COVID-19 Daily Positive Infections: State of " + s 

#plt.annotate(s +" 3-day: "+ states[states["state"]==s ]["3 day moving"].iloc[(-1)].round(3).astype(str),  color=c,size=20, 
#             xy=(0.15, y), 
#             xytext=(0.15, y),horizontalalignment = 'left',
#             xycoords = 'figure fraction',textcoords='figure fraction')
y+=0.025
plt.title(title, fontsize=24)
plt.xlabel('Date', fontsize=20)
plt.ylabel('Daily New Cases of COVID-19', fontsize=24)
plt.rcParams['xtick.labelsize'] =22
plt.figtext(0.90, 0.01, 'Data gathered by Johns Hopkins University https://github.com/CSSEGISandData ' + now, horizontalalignment='right')
plt.figtext(0.10, 0.01, 'David A. West, IBM')
plt.tick_params(axis='both', which='minor', labelsize=22)
plt.tick_params(axis='both', which='major', labelsize=22)
plt.grid(which='major',  linestyle='--',linewidth='1')
#plt.grid(which='major', axis='both', linestyle='--',linewidth='1')
#plt.grid(b=True, which='minor', axis='both', linestyle=':', linewidth='2', color='black')

matplotlib.ticker.AutoMinorLocator(n=2)
plt.minorticks_on()
plt.grid( b=True,which='minor', axis='x', color='black', linestyle='-',alpha=0.1)
plt.xticks(fontsize=14, rotation=0)

fn = s + '_daily_case_count_' + nowdate +'.jpg'
plt.savefig(fn)


plt.show()

# Write to CSV

masterfn = "Master_Data_" + str(new_start_date) + ".csv"
print(masterfn)
master.to_csv(masterfn)
m2.to_csv("Master_Data_no_State_Correction" + str(new_start_date) + ".csv")